# <span style="color:white; font-family:PT Sans Narrow; font-size:1.1em"> Import Packages </span>

In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys 
data_dir = os.path.dirname(os.path.realpath('.'))
sys.path.append(data_dir)


from crowdkit.datasets import load_dataset
from crowdkit import aggregation
import crowdkit
import pandas as pd
import ipywidgets
import load_data 
import funcs 
from sklearn import metrics
from classification_benchmark_modified import NistTrecRelevance

print('data_dir:', data_dir)

%reload_ext load_data
%reload_ext funcs
%reload_ext classification_benchmark_modified

2021-12-15 20:51:33.414330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


data_dir: /home/u29/mohammadsmajdi/projects/chest_xray/crowd_kit


2021-12-15 20:51:41.727906: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-15 20:51:41.729155: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-15 20:51:41.729191: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-15 20:51:41.729228: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sdmz-dtn-3.hpc.arizona.edu): /proc/driver/nvidia/version does not exist
2021-12-15 20:51:41.729808: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other op

# <span style="color:white; opacity:0.7; font-family:PT Sans Narrow; font-size:1.3em"> 1. Loading & Pre-processing of Data </span>
## <span style="color:green; font-family:PT Sans Narrow; font-size:1.1em"> 1.1 Selecting the Dataset </span>


List of all datasets:
- ('1. kr-vs-kp' ) 
- ('2. mushroom' )
- ('3. sick'     )
- ('4. spambase' )
- ('5. tic-tac-toe' )
- ('6. splice'      )
- ('8. waveform'    )
- ('9. biodeg'      )
- ('10. horse-colic')
- ('11. ionosphere' )
- ('12. vote'       )


In [2]:
WHICH_DATASET = 'ionosphere'
data, feature_columns = load_data.aim1_3_read_download_UCI_database(WHICH_DATASET=WHICH_DATASET, mode='read')

print('train shape:',data['train'].shape)
data['train'].head(3)

directory found /home/u29/mohammadsmajdi/projects/chest_xray/crowd_kit/data_mine
train shape: (281, 35)


,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a25,a26,a27,a28,a29,a30,a31,a32,a33,true
1,1,0,1.0,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.1904,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1,0,1.0,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.4310,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1,0,1.0,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.0000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0


## <span style="color:green; font-family:PT Sans Narrow; font-size:1.1em"> 1.2 Finding the noisy labels for each worker </span>

In [3]:
ARLS = {'num_labelers': 10, 
        'low_dis':      0.3, 
        'high_dis':     0.9}

predicted_labels, uncertainty, true_labels, labelers_strength = funcs.apply_technique_aim_1_3( data = data, ARLS = ARLS, num_simulations = 20,  feature_columns = feature_columns)

workers: 100%|██████████| 10/10 [00:02<00:00,  3.87it/s]


# <span style="color:white; opacity:0.7; font-family:PT Sans Narrow; font-size:1.3em"> 2. Applying the state of art techniques to the data </span>

In [21]:
NTR = NistTrecRelevance(true_labels=true_labels['train'] , num_labelers=ARLS['num_labelers'])
NTR.apply_all_benchmarks()
NTR.accuracy.T

num_labelers,10
GoldMajorityVote,0.825623
MajorityVote,0.711744
MMSR,0.900356
Wawa,0.793594
ZeroBasedSkill,0.797153
GLAD,0.911032
DawidSkene,0.900356
